<a href="https://colab.research.google.com/github/Jasper-Hewitt/Wind_project/blob/main/mom324.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# init

## get and split data

In [ ]:
#todo
#fix the max number of tokens, it's actually very little
#補充 answer with the rest of the rows
#make a better while loop that creates different dataframes based on word count, code now a bit ugly
!pip install fitz
!pip install static
!pip install frontend
!pip install pymupdf --upgrade
!pip install openai
!pip install requests

In [2]:
import os
import openai
import json

openai.api_key = '' #delete when uploading 

In [3]:
#importing it doesn't work, fitz will say it is a 'broken document'. Better drag it in from dashboard.

# import requests

# url = "https://github.com/Jasper-Hewitt/Wind_project/blob/main/data/mom_324.pdf"
# response = requests.get(url)

# with open("file.pdf", "wb") as f:
#     f.write(response.content)


In [4]:
#manually download the data from github and upload it in this notebook
import fitz

doc = fitz.open("mom_324.pdf")

for page in doc:
    text = page.get_text("text")
    with open("output3.txt", "a") as f:
        f.write(text)

In [ ]:
with open('output3.txt', 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

In [6]:
import re
pattern = "。" #。|:
split_contents = re.split(pattern, contents)
# print(sentences)

In [ ]:
split_contents

In [8]:
len(split_contents)

435

# question 1: 第324次會議是什麼時候？


## sentiment search 1： 第327次會議是什麼時候？

In [ ]:
#sentiment search
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# add queries, between 3 and 5
queries = ['第324次會議是什麼時候？'] #'第327次會議中，對於鯨豚保育的重點是什麼？', '第327次會議中，有哪些專家為鯨豚保育發言？']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("Sentence: ", corpus[idx], " Original Sequence: ", idx+1, " (Score: {:.4f})".format(score))

In [10]:
len(queries)

1

In [11]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [12]:
len(sentence_scores)

426

In [13]:
#adjust to also allow for queries outside the range of 3-5
import pandas as pd

if len(queries)==1:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]]))

elif len(queries)==2:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[0]]+scores[queries[1]]))

elif len(queries)==3:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]))
elif len(queries)==4:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]))
elif len(queries)==5:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[4]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]+scores[queries[4]])) 

#________________________________________________________________________________________________________________________________________________________________________________________________________

if len(queries) == 1:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 2:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 3:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 4:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 5:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'similarity_score5', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

In [14]:
df

,text,original_sequence,accumulated_score
2,\n陸、 確認本會第 323 次會議紀錄 \n結論：第 323 次會議紀錄確認,3,"tensor(0.7544, device='cuda:0')"
0,\n1 \n行政院環境保護署環境影響評估審查委員會 \n第 324 次會議紀錄 \n壹、 ...,1,"tensor(0.5829, device='cuda:0')"
25,\n（三） 106 年 10 月 31 日專案小組初審會議結論（二）及前述\n修正意見併提...,26,"tensor(0.5722, device='cuda:0')"
42,\n（三） 106 年 10 月 24 日專案小組第 2 次初審會議結論（一）\n及前述修...,43,"tensor(0.5614, device='cuda:0')"
66,\n（三） 106 年 12 月 1 日專案小組初審會議結論（一）及前述\n修正意見併提委...,67,"tensor(0.5583, device='cuda:0')"
...,...,...,...
193,各項目評估結果均符合標準，且影響\n範圍侷限於場址附近，對其他國家之環境無造成顯\n著不利影響,194,"tensor(-0.0782, device='cuda:0')"
156,各項目評估結果均符合標準，且影響範圍侷限於場\n址附近，對其他國家之環境無造成顯著不利影響,157,"tensor(-0.0825, device='cuda:0')"
401,\n八、空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高級...,402,"tensor(-0.0826, device='cuda:0')"
421,\n哺乳類、兩棲類、爬蟲類、蝶類、蜻蜓等調查皆無保育\n類,422,"tensor(-0.0983, device='cuda:0')"


In [15]:
noise_pattern= '遵照辦理'
noise_pattern1='章節'
noise_pattern2='��'
noise_pattern3='�'
noise_pattern4='修訂處'
noise_pattern5='頁次'

df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern1,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern4,'',regex=True)
df['text'] = df['text'].replace(noise_pattern5,'',regex=True)

In [16]:
#delete empty rows
df = df[df['text'].str.len()>6]   
#the line below no longer makes sense lol. 
# df = df[['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df[['text', 'original_sequence', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,original_sequence,accumulated_score
0,\n陸、 確認本會第 323 次會議紀錄 \n結論：第 323 次會議紀錄確認,3,"tensor(0.7544, device='cuda:0')"
1,\n1 \n行政院環境保護署環境影響評估審查委員會 \n第 324 次會議紀錄 \n壹、 ...,1,"tensor(0.5829, device='cuda:0')"
2,\n（三） 106 年 10 月 31 日專案小組初審會議結論（二）及前述\n修正意見併提...,26,"tensor(0.5722, device='cuda:0')"
3,\n（三） 106 年 10 月 24 日專案小組第 2 次初審會議結論（一）\n及前述修...,43,"tensor(0.5614, device='cuda:0')"
4,\n（三） 106 年 12 月 1 日專案小組初審會議結論（一）及前述\n修正意見併提委...,67,"tensor(0.5583, device='cuda:0')"
...,...,...,...
421,各項目評估結果均符合標準，且影響\n範圍侷限於場址附近，對其他國家之環境無造成顯\n著不利影響,194,"tensor(-0.0782, device='cuda:0')"
422,各項目評估結果均符合標準，且影響範圍侷限於場\n址附近，對其他國家之環境無造成顯著不利影響,157,"tensor(-0.0825, device='cuda:0')"
423,\n八、空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高級...,402,"tensor(-0.0826, device='cuda:0')"
424,\n哺乳類、兩棲類、爬蟲類、蝶類、蜻蜓等調查皆無保育\n類,422,"tensor(-0.0983, device='cuda:0')"


In [17]:
#I used sentiment search to judge whether GPTs output is related to our keywords but I ran into a problem: there is no way to check whether the information is complete. For example, if we set the default threshold
#to 0.7000, and then ask chatGPT to make a summary about certain topics within that text, it will only make a summary about those topics and leave the rest#####
#also, if start with a really high threshold (which may only lead to 2 or 3 sentences) chat gpt will write a perfect summary that SBERT will judge with a high tensor score. This means that the threshold will not go down
#and much information that may also be related to the subject is being left out.

#I therefore thought of an alternative way to create a dynamic threshold. In the code below we find the last row, so the lowest possible tensor score, that contains our 關鍵詞 and delete all the rows after that.
#this way we can be sure that all the important sentences related to our topic will be taken into consideration by GPT for writing its summary or answering our questions. 

# Find the index of the last row containing the word "鯨豚"
index = df[df['text'].str.contains('會議')][::-1].index[0]

# Drop all of the rows after the last row containing the word "鯨豚"
new_df = df.loc[:index]

In [18]:
new_df

,text,original_sequence,accumulated_score
0,\n陸、 確認本會第 323 次會議紀錄 \n結論：第 323 次會議紀錄確認,3,"tensor(0.7544, device='cuda:0')"
1,\n1 \n行政院環境保護署環境影響評估審查委員會 \n第 324 次會議紀錄 \n壹、 ...,1,"tensor(0.5829, device='cuda:0')"
2,\n（三） 106 年 10 月 31 日專案小組初審會議結論（二）及前述\n修正意見併提...,26,"tensor(0.5722, device='cuda:0')"
3,\n（三） 106 年 10 月 24 日專案小組第 2 次初審會議結論（一）\n及前述修...,43,"tensor(0.5614, device='cuda:0')"
4,\n（三） 106 年 12 月 1 日專案小組初審會議結論（一）及前述\n修正意見併提委...,67,"tensor(0.5583, device='cuda:0')"
...,...,...,...
179,\n三、彰化縣政府（農業處） \n（一）回覆之補充、修正資料所提「鯨豚觀察員需定時交換位\...,336,"tensor(0.1500, device='cuda:0')"
180,」 \n（二） 本署環境督察總隊發言如附件 2，吳委員義林詢問略以：\n「簡報 P.11 環...,46,"tensor(0.1495, device='cuda:0')"
181,」 \n（五） 開發單位綜合回覆說明如附件 10,168,"tensor(0.1494, device='cuda:0')"
182,\n（2）離岸風力發電機組施工期水下噪音評估方法及閾\n值，除配合經濟部能源局所提任務小組...,118,"tensor(0.1492, device='cuda:0')"


### determine threshold

In [19]:
#we will have to consider how we want to determine the threshold, the simplest way would be to look at the dataframe and check from where it doesn't have anything to do 
#with 噪音 anymore. if there is a more mathematical way to solve this problem I am also open to that. 

# threshold = 0.7710
# mask = df['accumulated_score'].apply(lambda x: x.item() > threshold)

# sbert_out = df[mask]
# sbert_out

### sort by original sequence (optional)

In [20]:
# sbert_out = sbert_out.sort_values(by='original_sequence').reset_index(drop=True)
# sbert_out=sbert_out['text']

In [21]:
# sbert_out=sbert_out.to_frame()
# sbert_out

## Summary with GPT 

### prepare data

In [22]:
#disable this cell for sort by orginal sequence 
#fix this this is super confusing lol!!!!
sbert_out=new_df
sbert_out=sbert_out['text']  
sbert_out=sbert_out.to_frame() 
df=sbert_out

In [23]:
df      

,text
0,\n陸、 確認本會第 323 次會議紀錄 \n結論：第 323 次會議紀錄確認
1,\n1 \n行政院環境保護署環境影響評估審查委員會 \n第 324 次會議紀錄 \n壹、 ...
2,\n（三） 106 年 10 月 31 日專案小組初審會議結論（二）及前述\n修正意見併提...
3,\n（三） 106 年 10 月 24 日專案小組第 2 次初審會議結論（一）\n及前述修...
4,\n（三） 106 年 12 月 1 日專案小組初審會議結論（一）及前述\n修正意見併提委...
...,...
179,\n三、彰化縣政府（農業處） \n（一）回覆之補充、修正資料所提「鯨豚觀察員需定時交換位\...
180,」 \n（二） 本署環境督察總隊發言如附件 2，吳委員義林詢問略以：\n「簡報 P.11 環...
181,」 \n（五） 開發單位綜合回覆說明如附件 10
182,\n（2）離岸風力發電機組施工期水下噪音評估方法及閾\n值，除配合經濟部能源局所提任務小組...


In [24]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 1500
index = df.index[df['cumulative_length'] > 1500]

if index.size > 0:
    index = index[0] - 1
else:
    index = df.shape[0] - 1

# Create a new dataframe with only the last few rows
new_df = df.iloc[index+1:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index+1]
df

,text,text_length,cumulative_length
0,\n陸、 確認本會第 323 次會議紀錄 \n結論：第 323 次會議紀錄確認,38,38
1,\n1 \n行政院環境保護署環境影響評估審查委員會 \n第 324 次會議紀錄 \n壹、 ...,170,208
2,\n（三） 106 年 10 月 31 日專案小組初審會議結論（二）及前述\n修正意見併提...,49,257
3,\n（三） 106 年 10 月 24 日專案小組第 2 次初審會議結論（一）\n及前述修...,54,311
4,\n（三） 106 年 12 月 1 日專案小組初審會議結論（一）及前述\n修正意見併提委...,48,359
5,」 \n（三） 106 年 10 月 3 日專案小組第 2 次初審會議結論（一）\n及前述修...,54,413
6,\n伍、 確認出席委員已達法定人數後，主席致詞：略,25,438
7,\n（二）P.審-5、審-12 辦理情形提及：「故本球場擴增面積為\n18 洞場地乙案，業...,216,654
8,\n（二） 開發單位於 106 年 12 月 21 日函送補正資料至本署，業\n經本署轉送...,60,714
9,」 \n（三） 內政部代表發言略以：「本案於本部區域計畫委員會進\n行 2 次專案小組審查，...,111,825


In [26]:
sbert_out=df['text']
# sbert_out_1=new_df['text']
# sbert_out_1 = sbert_out_1.reset_index(drop=True)
# sbert_out.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
# sbert_out_1.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
sbert_out=sbert_out.to_frame()


In [27]:
sbert_out

,text
0,\n陸、 確認本會第 323 次會議紀錄 \n結論：第 323 次會議紀錄確認
1,\n1 \n行政院環境保護署環境影響評估審查委員會 \n第 324 次會議紀錄 \n壹、 ...
2,\n（三） 106 年 10 月 31 日專案小組初審會議結論（二）及前述\n修正意見併提...
3,\n（三） 106 年 10 月 24 日專案小組第 2 次初審會議結論（一）\n及前述修...
4,\n（三） 106 年 12 月 1 日專案小組初審會議結論（一）及前述\n修正意見併提委...
5,」 \n（三） 106 年 10 月 3 日專案小組第 2 次初審會議結論（一）\n及前述修...
6,\n伍、 確認出席委員已達法定人數後，主席致詞：略
7,\n（二）P.審-5、審-12 辦理情形提及：「故本球場擴增面積為\n18 洞場地乙案，業...
8,\n（二） 開發單位於 106 年 12 月 21 日函送補正資料至本署，業\n經本署轉送...
9,」 \n（三） 內政部代表發言略以：「本案於本部區域計畫委員會進\n行 2 次專案小組審查，...


In [28]:
# sbert_text = sbert_out['text'].to_string(index=False) #this leads to incomplete text. use join
sbert_text = " ".join(sbert_out['text'].tolist())
len(sbert_text)

1462

In [29]:
sbert_text = re.sub(r'[\n\\]+', '', sbert_text)
sbert_text= re.sub(r'n|n－', '', sbert_text)
sbert_text

' 陸、 確認本會第 323 次會議紀錄 結論：第 323 次會議紀錄確認  1 行政院環境保護署環境影響評估審查委員會 第 324 次會議紀錄 壹、 107 年 1 月 17 日（星期三）下午 2 時 0 分 貳、 地點：本署 4 樓第 5 會議室 參、 主席：李主任委員應元（詹副主任委員順貴代）                     記錄：商維庭 肆、 出（列）席單位及人員：如後附會議簽名單  （三） 106 年 10 月 31 日專案小組初審會議結論（二）及前述修正意見併提委員會討論  （三） 106 年 10 月 24 日專案小組第 2 次初審會議結論（一）及前述修正意見併提委員會討論  （三） 106 年 12 月 1 日專案小組初審會議結論（一）及前述修正意見併提委員會討論 」 （三） 106 年 10 月 3 日專案小組第 2 次初審會議結論（一）及前述修正意見併提委員會討論  伍、 確認出席委員已達法定人數後，主席致詞：略  （二）P.審-5、審-12 辦理情形提及：「故本球場擴增面積為18 洞場地乙案，業於 103 年 12 月 4 日獲教育部體育署以臺教授體字第 1030035856 號函同意…」 七、內政部營建署 其他：本案本署所提意見（P.審－20）係供行政院環境保護署審查參考，本案已提本部區域計畫委員會 106 年 12 月 28 日第 2 次專案小組會議審查，後續申請人尚需依專案小組會議審查意見補正後提區域計畫委員會審議  （二） 開發單位於 106 年 12 月 21 日函送補正資料至本署，業經本署轉送有關委員、專家學者及相關機關確認 」 （三） 內政部代表發言略以：「本案於本部區域計畫委員會進行 2 次專案小組審查，目前程序仍在進行，本案於 106年 12 月 28 日專案小組第 2 次審查會議，開發單位後續補充資料後，提本部區域計畫委員會討論        行政院環境保護署環境影響評估審查委員會第 324 次會議列席單位、旁聽民眾團體發言單或書面意見   捌、 臨時提案 玖、 散會（下午 7 時 40 分）  （二） 開發單位於 106 年 12 月 27 日函送補正資料至本署，經本署轉送有關委員、專家學者及相關機關確認，惟王委員价巨、李委員堅明、劉委員益昌、王委員文誠、林教授慶偉、教育部、內政部營建署、文化部文化資產局、臺中市政

In [30]:
len(sbert_text)

1395

### prompt GPT

In [42]:
summary = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '第324次會議是幾年幾月幾日？'+sbert_text,
  temperature=0.9,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary = summary + text

summary

'，但仍懇請臺中市政府環境保護局歸還函上簽章確認之 」\n\n第324次會議是107年1月17日（星期三）下午2時0分。'

In [43]:
print('第324次會議是什麼時候？'+ summary)
# print('第327次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
# print('第327次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第324次會議是什麼時候？，但仍懇請臺中市政府環境保護局歸還函上簽章確認之 」

第324次會議是107年1月17日（星期三）下午2時0分。


# question 2: 第327次會議中，對於鯨豚保育的重點是什麼？

## sentiment search 2

In [ ]:
#sentiment search
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# add queries, between 1 and 5
queries = ['第324次會議中，對於鯨豚保育的重點是什麼？'] #, '第327次會議中，有哪些專家為鯨豚保育發言？']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("Sentence: ", corpus[idx], " Original Sequence: ", idx+1, " (Score: {:.4f})".format(score))

In [45]:
type(queries)

list

In [46]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [47]:
len(sentence_scores)

426

In [48]:
#adjust to also allow for queries outside the range of 3-5
import pandas as pd

if len(queries)==1:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]]))

elif len(queries)==2:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[0]]+scores[queries[1]]))

elif len(queries)==3:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]))
elif len(queries)==4:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]))
elif len(queries)==5:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[4]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]+scores[queries[4]])) 

#________________________________________________________________________________________________________________________________________________________________________________________________________

if len(queries) == 1:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 2:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 3:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 4:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 5:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'similarity_score5', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

In [49]:
df

,text,original_sequence,accumulated_score
122,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...,123,"tensor(0.6937, device='cuda:0')"
374,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕...",375,"tensor(0.6929, device='cuda:0')"
143,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...,144,"tensor(0.6843, device='cuda:0')"
180,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,181,"tensor(0.6769, device='cuda:0')"
381,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...,382,"tensor(0.6720, device='cuda:0')"
...,...,...,...
317,營運期間主要空氣污染物來源\n為交通車輛行駛所產生的廢氣排放，以空氣品質擴散模\n式分析其對...,318,"tensor(0.0369, device='cuda:0')"
321,本案施工振動主要來自振動較大之\n施工機具包括挖土機等；道路交通振動則由運送重型機\n具設備...,322,"tensor(0.0261, device='cuda:0')"
123,另日\n落前 2 小時後不得啟動新設風機打樁作業,124,"tensor(0.0247, device='cuda:0')"
401,\n八、空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高級...,402,"tensor(0.0240, device='cuda:0')"


In [50]:
noise_pattern= '遵照辦理'
noise_pattern1='章節'
noise_pattern2='��'
noise_pattern3='�'
noise_pattern4='修訂處'
noise_pattern5='頁次'

df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern1,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern4,'',regex=True)
df['text'] = df['text'].replace(noise_pattern5,'',regex=True)

In [51]:
#delete empty rows
df = df[df['text'].str.len()>6]   
#the line below no longer makes sense lol. 
# df = df[['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df[['text', 'original_sequence', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,original_sequence,accumulated_score
0,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...,123,"tensor(0.6937, device='cuda:0')"
1,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕...",375,"tensor(0.6929, device='cuda:0')"
2,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...,144,"tensor(0.6843, device='cuda:0')"
3,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,181,"tensor(0.6769, device='cuda:0')"
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...,382,"tensor(0.6720, device='cuda:0')"
...,...,...,...
421,營運期間主要空氣污染物來源\n為交通車輛行駛所產生的廢氣排放，以空氣品質擴散模\n式分析其對...,318,"tensor(0.0369, device='cuda:0')"
422,本案施工振動主要來自振動較大之\n施工機具包括挖土機等；道路交通振動則由運送重型機\n具設備...,322,"tensor(0.0261, device='cuda:0')"
423,另日\n落前 2 小時後不得啟動新設風機打樁作業,124,"tensor(0.0247, device='cuda:0')"
424,\n八、空氣污染 \n施工階段引擎應優先採用低硫氧化物及粒狀污染物等空氣污\n染排放之高級...,402,"tensor(0.0240, device='cuda:0')"


In [52]:
#I used sentiment search to judge whether GPTs output is related to our keywords but I ran into a problem: there is no way to check whether the information is complete. For example, if we set the default threshold
#to 0.7000, and then ask chatGPT to make a summary about certain topics within that text, it will only make a summary about those topics and leave the rest#####
#also, if start with a really high threshold (which may only lead to 2 or 3 sentences) chat gpt will write a perfect summary that SBERT will judge with a high tensor score. This means that the threshold will not go down
#and much information that may also be related to the subject is being left out.

#I therefore thought of an alternative way to create a dynamic threshold. In the code below we find the last row, so the lowest possible tensor score, that contains our 關鍵詞 and delete all the rows after that.
#this way we can be sure that all the important sentences related to our topic will be taken into consideration by GPT for writing its summary or answering our questions. 

# Find the index of the last row containing the word "鯨豚"
index = df[df['text'].str.contains('鯨豚')][::-1].index[0]

# Drop all of the rows after the last row containing the word "鯨豚"
new_df = df.loc[:index]

In [53]:
new_df

,text,original_sequence,accumulated_score
0,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...,123,"tensor(0.6937, device='cuda:0')"
1,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕...",375,"tensor(0.6929, device='cuda:0')"
2,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...,144,"tensor(0.6843, device='cuda:0')"
3,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,181,"tensor(0.6769, device='cuda:0')"
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...,382,"tensor(0.6720, device='cuda:0')"
...,...,...,...
298,\n4. 調查開發區內移植樹木之樹高、胸徑、樹種等，補充樹\n木移植計畫（含移植地點、樹木...,109,"tensor(0.2431, device='cuda:0')"
299,\n（4）綜整本計畫對當地環境之影響結果顯示，本案開發\n未使當地環境逾越環境品質標準或超...,186,"tensor(0.2430, device='cuda:0')"
300,開發單位回覆\n \n12 \n本案可為衝擊性相當低之開發，因高爾夫球場開發已改\n變整個植...,94,"tensor(0.2422, device='cuda:0')"
301,\n8. 評估土壤沖蝕滲透及地下水補注之改變,113,"tensor(0.2419, device='cuda:0')"


## Summary with GPT 2

### prepare data

In [54]:
#disable this cell for sort by orginal sequence 
#fix this this is super confusing lol!!!!
sbert_out=new_df
sbert_out=sbert_out['text']  
sbert_out=sbert_out.to_frame() 
df=sbert_out

In [55]:
df      

,text
0,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...
1,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕..."
2,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...
3,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...
...,...
298,\n4. 調查開發區內移植樹木之樹高、胸徑、樹種等，補充樹\n木移植計畫（含移植地點、樹木...
299,\n（4）綜整本計畫對當地環境之影響結果顯示，本案開發\n未使當地環境逾越環境品質標準或超...
300,開發單位回覆\n \n12 \n本案可為衝擊性相當低之開發，因高爾夫球場開發已改\n變整個植...
301,\n8. 評估土壤沖蝕滲透及地下水補注之改變


In [56]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 1500
index = df.index[df['cumulative_length'] > 1500]

if index.size > 0:
    index = index[0] - 1
else:
    index = df.shape[0] - 1

# Create a new dataframe with only the last few rows
new_df = df.iloc[index+1:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index+1]
df

,text,text_length,cumulative_length
0,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...,96,96
1,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕...",97,193
2,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...,60,253
3,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,61,314
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...,93,407
5,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,69,476
6,\n三、彰化縣政府（農業處） \n（一）回覆之補充、修正資料所提「鯨豚觀察員需定時交換位\...,209,685
7,\n3. 施工營運期間 4艘觀測船各船配置至少2名以上鯨豚觀\n測員中，同一時間應至少有 ...,71,756
8,\n（二）針對前項特定保護對象需納入緩衝帶規劃，建議基座位\n址需距離中華白海豚野生動物重...,65,821
9,基於\n環境衝擊之考量，本計畫風機位置已避開白海豚重要棲息區\n域、重要野鳥棲地、螻蛄蝦繁殖...,74,895


In [57]:
df

,text,text_length,cumulative_length
0,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...,96,96
1,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕...",97,193
2,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...,60,253
3,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,61,314
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...,93,407
5,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,69,476
6,\n三、彰化縣政府（農業處） \n（一）回覆之補充、修正資料所提「鯨豚觀察員需定時交換位\...,209,685
7,\n3. 施工營運期間 4艘觀測船各船配置至少2名以上鯨豚觀\n測員中，同一時間應至少有 ...,71,756
8,\n（二）針對前項特定保護對象需納入緩衝帶規劃，建議基座位\n址需距離中華白海豚野生動物重...,65,821
9,基於\n環境衝擊之考量，本計畫風機位置已避開白海豚重要棲息區\n域、重要野鳥棲地、螻蛄蝦繁殖...,74,895


In [58]:
sbert_out=df['text']
# sbert_out_1=new_df['text']
# sbert_out_1 = sbert_out_1.reset_index(drop=True)
# sbert_out.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
# sbert_out_1.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
sbert_out=sbert_out.to_frame()


In [59]:
sbert_out

,text
0,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...
1,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕..."
2,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...
3,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...
5,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...
6,\n三、彰化縣政府（農業處） \n（一）回覆之補充、修正資料所提「鯨豚觀察員需定時交換位\...
7,\n3. 施工營運期間 4艘觀測船各船配置至少2名以上鯨豚觀\n測員中，同一時間應至少有 ...
8,\n（二）針對前項特定保護對象需納入緩衝帶規劃，建議基座位\n址需距離中華白海豚野生動物重...
9,基於\n環境衝擊之考量，本計畫風機位置已避開白海豚重要棲息區\n域、重要野鳥棲地、螻蛄蝦繁殖...


In [60]:
# sbert_text = sbert_out['text'].to_string(index=False) #this leads to incomplete text. use join
sbert_text = " ".join(sbert_out['text'].tolist())
len(sbert_text)

1460

In [61]:
sbert_text = re.sub(r'[\n\\]+', '', sbert_text)
sbert_text= re.sub(r'n|n－', '', sbert_text)
sbert_text

' （4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4 月至 9 月執行，調整前應依法申請變更），建議強化鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能力人員共同參與  （2）環境影響評估書件記載風場區域及外圍 1,500 公尺發現有鯨豚母子對或瀕臨絕種類保育類鯨豚之案件，應繼續辦理 4 季合計至少 30 趟次之鯨豚調查作業，並提出環境影響調查報告送審  （4）鯨豚：本計畫風場非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模擬評估結果，已擬定海豚保護措施  ○4 鯨豚：本計畫風場非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模擬評估結果，已擬定海豚保護措施  （七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4~9 月執行，調整前應依法申請變更），建議強化鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能力人員共同參與  3. 行政院農業委員會依野生動物保育法訂定之「野生動物重要棲息環境」「野生動物保護區」「中華白海豚野生動物重要棲息環境（含預告）」  三、彰化縣政府（農業處） （一）回覆之補充、修正資料所提「鯨豚觀察員需定時交換位置，每個人輪替完不同觀察位置後，交換到休息位置休息....」，惟第八章 P.8-2 所述，每船僅備置 2 名鯨豚觀察員，1 人觀察警戒區，1 人觀察預警區，似難有休息之虞，為免鯨豚觀察員長時間工作影響監測狀況之判斷，每船配置數量仍請再考量，並建議補充「至少」配置人數，避免屆時數量不足時卻因環境影響差異分析程序延誤期程  3. 施工營運期間 4艘觀測船各船配置至少2名以上鯨豚觀測員中，同一時間應至少有 1 位為民間生態團體成員，於基礎打樁過程同時目視觀察  （二）針對前項特定保護對象需納入緩衝帶規劃，建議基座位址需距離中華白海豚野生動物重要棲息環境（含預告）1,000 公尺以上 基於環境衝擊之考量，本計畫風機位置已避開白海豚重要棲息區域、重要野鳥棲地、螻蛄蝦繁殖保育區、漢寶溼地、保護礁區、航道、軍事禁限建及相關開發計畫  （五）就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等保育類野生動物之規劃，恐衍生疑慮，建議暫緩採用，宜審慎蒐集案例研析後再行考量  2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察船及配置鯨豚生態觀察員，於基礎打樁過程持續監測  3. 開發單位就專案小組

In [62]:
len(sbert_text)

1394

### prompt GPT

In [64]:
summary1 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= ' 第324次會議中，對於鯨豚保育的重點是什麼？'+sbert_text,
  temperature=0.9,
  max_tokens=900,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary1 = summary1 + text

summary1

'  第324次會議中，對於鯨豚保育的重點是避免對中華白海豚野生動物的衝擊，包括：施工前使用聲音驅趕裝置暫時驅趕野生動物，風機位置需距離中華白海豚野生動物重要棲息環境（含預告）1,000 公尺以上，開發單位承諾退縮風場至距離中華白海豚重要棲息環境至少 4.2 公里、水深大於 37 公尺（TWVD2001 為基準）區域，於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察船及配置鯨豚生態觀察員，於環境影響評估內容中，呈現最保守情境再加上極端氣候之最壞情境下評估，以及船隻船速管制在 6 節以下，且儘可能避免在中華白海豚活動高峰時間進入已知之中華白海豚活動密集位置，航道劃設也應避開敏感區位。'

In [65]:
#print('第327次會議是什麼時候？'+ summary)
print('第324次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
# print('第327次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第324次會議中，對於鯨豚保育的重點是什麼？  第324次會議中，對於鯨豚保育的重點是避免對中華白海豚野生動物的衝擊，包括：施工前使用聲音驅趕裝置暫時驅趕野生動物，風機位置需距離中華白海豚野生動物重要棲息環境（含預告）1,000 公尺以上，開發單位承諾退縮風場至距離中華白海豚重要棲息環境至少 4.2 公里、水深大於 37 公尺（TWVD2001 為基準）區域，於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察船及配置鯨豚生態觀察員，於環境影響評估內容中，呈現最保守情境再加上極端氣候之最壞情境下評估，以及船隻船速管制在 6 節以下，且儘可能避免在中華白海豚活動高峰時間進入已知之中華白海豚活動密集位置，航道劃設也應避開敏感區位。


#question 3:  第327次會議中，有哪些專家為鯨豚保育發言？

## sentiment search 3

In [ ]:
#sentiment search
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# add queries, between 3 and 5
queries = ['第324次會議中，有哪些專家為鯨豚保育發言？']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("Sentence: ", corpus[idx], " Original Sequence: ", idx+1, " (Score: {:.4f})".format(score))

In [67]:
len(queries)

1

In [68]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [69]:
len(sentence_scores)

426

In [70]:
#adjust to also allow for queries outside the range of 3-5
import pandas as pd

if len(queries)==1:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]]))

elif len(queries)==2:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[0]]+scores[queries[1]]))

elif len(queries)==3:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]))
elif len(queries)==4:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]))
elif len(queries)==5:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[4]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]+scores[queries[4]])) 

#________________________________________________________________________________________________________________________________________________________________________________________________________

if len(queries) == 1:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 2:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 3:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 4:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 5:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'similarity_score5', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

In [71]:
df

,text,original_sequence,accumulated_score
374,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕...",375,"tensor(0.6813, device='cuda:0')"
122,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...,123,"tensor(0.6666, device='cuda:0')"
180,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,181,"tensor(0.6605, device='cuda:0')"
143,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...,144,"tensor(0.6585, device='cuda:0')"
381,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...,382,"tensor(0.6453, device='cuda:0')"
...,...,...,...
413,另細懸浮微粒(PM2.5)增量\n僅 0.032 μg/m3，惟受當地背景值為 37 μg/...,414,"tensor(0.0205, device='cuda:0')"
207,\n（5）維運中心優先購買使用市售已商業化電動車或混\n合動力車,208,"tensor(0.0196, device='cuda:0')"
320,模擬結果，營運階段運輸\n車輛所造成之噪音增量約為 0.3~1.6 dB(A)，屬於無影響\...,321,"tensor(0.0161, device='cuda:0')"
192,\n（7）本開發計畫係屬潔淨再生能源風力發電之開發計\n畫，營運階段於機組運轉期間僅以天然...,193,"tensor(0.0056, device='cuda:0')"


In [72]:
noise_pattern= '遵照辦理'
noise_pattern1='章節'
noise_pattern2='��'
noise_pattern3='�'
noise_pattern4='修訂處'
noise_pattern5='頁次'

df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern1,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern4,'',regex=True)
df['text'] = df['text'].replace(noise_pattern5,'',regex=True)

In [73]:
#delete empty rows
df = df[df['text'].str.len()>6]   
#the line below no longer makes sense lol. 
# df = df[['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df[['text', 'original_sequence', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,original_sequence,accumulated_score
0,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕...",375,"tensor(0.6813, device='cuda:0')"
1,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...,123,"tensor(0.6666, device='cuda:0')"
2,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,181,"tensor(0.6605, device='cuda:0')"
3,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...,144,"tensor(0.6585, device='cuda:0')"
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...,382,"tensor(0.6453, device='cuda:0')"
...,...,...,...
421,另細懸浮微粒(PM2.5)增量\n僅 0.032 μg/m3，惟受當地背景值為 37 μg/...,414,"tensor(0.0205, device='cuda:0')"
422,\n（5）維運中心優先購買使用市售已商業化電動車或混\n合動力車,208,"tensor(0.0196, device='cuda:0')"
423,模擬結果，營運階段運輸\n車輛所造成之噪音增量約為 0.3~1.6 dB(A)，屬於無影響\...,321,"tensor(0.0161, device='cuda:0')"
424,\n（7）本開發計畫係屬潔淨再生能源風力發電之開發計\n畫，營運階段於機組運轉期間僅以天然...,193,"tensor(0.0056, device='cuda:0')"


In [74]:
#I used sentiment search to judge whether GPTs output is related to our keywords but I ran into a problem: there is no way to check whether the information is complete. For example, if we set the default threshold
#to 0.7000, and then ask chatGPT to make a summary about certain topics within that text, it will only make a summary about those topics and leave the rest#####
#also, if start with a really high threshold (which may only lead to 2 or 3 sentences) chat gpt will write a perfect summary that SBERT will judge with a high tensor score. This means that the threshold will not go down
#and much information that may also be related to the subject is being left out.

#I therefore thought of an alternative way to create a dynamic threshold. In the code below we find the last row, so the lowest possible tensor score, that contains our 關鍵詞 and delete all the rows after that.
#this way we can be sure that all the important sentences related to our topic will be taken into consideration by GPT for writing its summary or answering our questions. 

# Find the index of the last row containing the word "鯨豚"
index = df[df['text'].str.contains('專家')][::-1].index[0]

# Drop all of the rows after the last row containing the word "鯨豚"
new_df = df.loc[:index]

In [75]:
new_df

,text,original_sequence,accumulated_score
0,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕...",375,"tensor(0.6813, device='cuda:0')"
1,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...,123,"tensor(0.6666, device='cuda:0')"
2,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,181,"tensor(0.6605, device='cuda:0')"
3,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...,144,"tensor(0.6585, device='cuda:0')"
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...,382,"tensor(0.6453, device='cuda:0')"
...,...,...,...
197,\n6. 文化部依水下文化資產保存法劃設之「水下文化資產保\n護區」與依法列冊及管理疑似水...,358,"tensor(0.3330, device='cuda:0')"
198,\n（5）本環境影響差異分析報告定稿備查後，變更內容\n \n5 \n始得實施,40,"tensor(0.3317, device='cuda:0')"
199,故專家小組最後決議建議本案認定不應開發，並提\n委員會討論,231,"tensor(0.3315, device='cuda:0')"
200,」\n研提積極推動管理方式,7,"tensor(0.3311, device='cuda:0')"


## Summary with GPT 3

### prepare data

In [76]:
#disable this cell for sort by orginal sequence 
#fix this this is super confusing lol!!!!
sbert_out=new_df
sbert_out=sbert_out['text']  
sbert_out=sbert_out.to_frame() 
df=sbert_out

In [77]:
df      

,text
0,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕..."
1,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...
2,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...
3,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...
...,...
197,\n6. 文化部依水下文化資產保存法劃設之「水下文化資產保\n護區」與依法列冊及管理疑似水...
198,\n（5）本環境影響差異分析報告定稿備查後，變更內容\n \n5 \n始得實施
199,故專家小組最後決議建議本案認定不應開發，並提\n委員會討論
200,」\n研提積極推動管理方式


In [78]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 1500
index = df.index[df['cumulative_length'] > 1500]

if index.size > 0:
    index = index[0] - 1
else:
    index = df.shape[0] - 1

# Create a new dataframe with only the last few rows
new_df = df.iloc[index+1:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index+1]
df

,text,text_length,cumulative_length
0,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕...",97,97
1,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...,96,193
2,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,61,254
3,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...,60,314
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...,93,407
5,\n三、彰化縣政府（農業處） \n（一）回覆之補充、修正資料所提「鯨豚觀察員需定時交換位\...,209,616
6,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,69,685
7,\n3. 施工營運期間 4艘觀測船各船配置至少2名以上鯨豚觀\n測員中，同一時間應至少有 ...,71,756
8,\n（五）就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群\n等保育類野生動物之規劃，恐衍生...,69,825
9,\n2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察\n船及配置鯨豚生態觀察員，...,55,880


In [79]:
df

,text,text_length,cumulative_length
0,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕...",97,97
1,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...,96,193
2,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...,61,254
3,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...,60,314
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...,93,407
5,\n三、彰化縣政府（農業處） \n（一）回覆之補充、修正資料所提「鯨豚觀察員需定時交換位\...,209,616
6,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...,69,685
7,\n3. 施工營運期間 4艘觀測船各船配置至少2名以上鯨豚觀\n測員中，同一時間應至少有 ...,71,756
8,\n（五）就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群\n等保育類野生動物之規劃，恐衍生...,69,825
9,\n2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察\n船及配置鯨豚生態觀察員，...,55,880


In [80]:
sbert_out=df['text']
# sbert_out_1=new_df['text']
# sbert_out_1 = sbert_out_1.reset_index(drop=True)
# sbert_out.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
# sbert_out_1.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
sbert_out=sbert_out.to_frame()


In [81]:
sbert_out

,text
0,"\n（2）環境影響評估書件記載風場區域及外圍 1,500 公尺\n發現有鯨豚母子對或瀕臨絕..."
1,\n（4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次\n（非僅限於 4 月至 9 月...
2,\n○\n4 鯨豚：本計畫風場非位於中華白海豚野生動物重要\n棲息環境預告範圍，並依水下噪...
3,\n（4）鯨豚：本計畫風場非位於中華白海豚野生動物重\n要棲息環境預告範圍，並依水下噪音模...
4,\n（七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非\n僅限於 4~9 月執行，...
5,\n三、彰化縣政府（農業處） \n（一）回覆之補充、修正資料所提「鯨豚觀察員需定時交換位\...
6,\n3. 行政院農業委員會依野生動物保育法訂定之「野生動物\n重要棲息環境」「野生動物保護...
7,\n3. 施工營運期間 4艘觀測船各船配置至少2名以上鯨豚觀\n測員中，同一時間應至少有 ...
8,\n（五）就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群\n等保育類野生動物之規劃，恐衍生...
9,\n2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察\n船及配置鯨豚生態觀察員，...


In [82]:
# sbert_text = sbert_out['text'].to_string(index=False) #this leads to incomplete text. use join
sbert_text = " ".join(sbert_out['text'].tolist())
len(sbert_text)

1496

In [83]:
sbert_text = re.sub(r'[\n\\]+', '', sbert_text)
sbert_text= re.sub(r'n|n－', '', sbert_text)
sbert_text

' （2）環境影響評估書件記載風場區域及外圍 1,500 公尺發現有鯨豚母子對或瀕臨絕種類保育類鯨豚之案件，應繼續辦理 4 季合計至少 30 趟次之鯨豚調查作業，並提出環境影響調查報告送審  （4）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4 月至 9 月執行，調整前應依法申請變更），建議強化鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能力人員共同參與  ○4 鯨豚：本計畫風場非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模擬評估結果，已擬定海豚保護措施  （4）鯨豚：本計畫風場非位於中華白海豚野生動物重要棲息環境預告範圍，並依水下噪音模擬評估結果，已擬定海豚保護措施  （七）施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4~9 月執行，調整前應依法申請變更），建議強化鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能力人員共同參與  三、彰化縣政府（農業處） （一）回覆之補充、修正資料所提「鯨豚觀察員需定時交換位置，每個人輪替完不同觀察位置後，交換到休息位置休息....」，惟第八章 P.8-2 所述，每船僅備置 2 名鯨豚觀察員，1 人觀察警戒區，1 人觀察預警區，似難有休息之虞，為免鯨豚觀察員長時間工作影響監測狀況之判斷，每船配置數量仍請再考量，並建議補充「至少」配置人數，避免屆時數量不足時卻因環境影響差異分析程序延誤期程  3. 行政院農業委員會依野生動物保育法訂定之「野生動物重要棲息環境」「野生動物保護區」「中華白海豚野生動物重要棲息環境（含預告）」  3. 施工營運期間 4艘觀測船各船配置至少2名以上鯨豚觀測員中，同一時間應至少有 1 位為民間生態團體成員，於基礎打樁過程同時目視觀察  （五）就施工前使用聲音驅趕裝置暫時驅趕中華白海豚族群等保育類野生動物之規劃，恐衍生疑慮，建議暫緩採用，宜審慎蒐集案例研析後再行考量  2. 於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察船及配置鯨豚生態觀察員，於基礎打樁過程持續監測  3. 開發單位就專案小組所提下列主要意見，已承諾納入辦理，請據以補充、修正環境影響說明書，經有關委員及相關機關確認後，提本署環境影響評估審查委員會討論： （1）開發單位承諾退縮風場至距離中華白海豚重要棲息 環 境 至 少 4.2 公 里 、 水 深 大 於 37 公 尺（TWVD2001 為基

In [84]:
len(sbert_text)

1430

### prompt GPT

In [90]:
 #remove key when uploading to github
summary2 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '第324次會議中，為鯨豚保育發言的專家有誰？'+sbert_text,
  temperature=0.9,
  max_tokens=800,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary2 = summary2 + text

summary2

'，提出全程使用減噪措施如水下氣泡幕、水下帷幕或施工時已商業化之「最佳」水下噪音防制工法，以及施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4~9 月執行，調整前應依法申請變更），建議強化鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能力人員共同參與等環境及生態保育相關施加措施。\n\n第324次會議中，為鯨豚保育發言的專家有：1. 公部門的農業處研究專家、2.環境影響評估書件專家、3.彰化縣政府（農業處）研究專家、4.行政院農業委員會專家、5.王委員文誠專家等。'

In [91]:
#print('第327次會議是什麼時候？'+ summary)
# print('第327次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
print('第324次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第324次會議中，有哪些專家為鯨豚保育發言？，提出全程使用減噪措施如水下氣泡幕、水下帷幕或施工時已商業化之「最佳」水下噪音防制工法，以及施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4~9 月執行，調整前應依法申請變更），建議強化鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能力人員共同參與等環境及生態保育相關施加措施。

第324次會議中，為鯨豚保育發言的專家有：1. 公部門的農業處研究專家、2.環境影響評估書件專家、3.彰化縣政府（農業處）研究專家、4.行政院農業委員會專家、5.王委員文誠專家等。


# final output (answers to all three questions)

In [92]:
print('第324次會議是什麼時候？'+ summary)
print('第324次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
print('第324次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第324次會議是什麼時候？，但仍懇請臺中市政府環境保護局歸還函上簽章確認之 」

第324次會議是107年1月17日（星期三）下午2時0分。
第324次會議中，對於鯨豚保育的重點是什麼？  第324次會議中，對於鯨豚保育的重點是避免對中華白海豚野生動物的衝擊，包括：施工前使用聲音驅趕裝置暫時驅趕野生動物，風機位置需距離中華白海豚野生動物重要棲息環境（含預告）1,000 公尺以上，開發單位承諾退縮風場至距離中華白海豚重要棲息環境至少 4.2 公里、水深大於 37 公尺（TWVD2001 為基準）區域，於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察船及配置鯨豚生態觀察員，於環境影響評估內容中，呈現最保守情境再加上極端氣候之最壞情境下評估，以及船隻船速管制在 6 節以下，且儘可能避免在中華白海豚活動高峰時間進入已知之中華白海豚活動密集位置，航道劃設也應避開敏感區位。
第324次會議中，有哪些專家為鯨豚保育發言？，提出全程使用減噪措施如水下氣泡幕、水下帷幕或施工時已商業化之「最佳」水下噪音防制工法，以及施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4~9 月執行，調整前應依法申請變更），建議強化鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能力人員共同參與等環境及生態保育相關施加措施。

第324次會議中，為鯨豚保育發言的專家有：1. 公部門的農業處研究專家、2.環境影響評估書件專家、3.彰化縣政府（農業處）研究專家、4.行政院農業委員會專家、5.王委員文誠專家等。


In [93]:
allQandA='第324次會議是什麼時候？'+summary +'第324次會議中，對於鯨豚保育的重點是什麼？'+ summary1+'第324次會議中，有哪些專家為鯨豚保育發言？'+ summary2

In [94]:
allQandA

'第324次會議是什麼時候？，但仍懇請臺中市政府環境保護局歸還函上簽章確認之 」\n\n第324次會議是107年1月17日（星期三）下午2時0分。第324次會議中，對於鯨豚保育的重點是什麼？  第324次會議中，對於鯨豚保育的重點是避免對中華白海豚野生動物的衝擊，包括：施工前使用聲音驅趕裝置暫時驅趕野生動物，風機位置需距離中華白海豚野生動物重要棲息環境（含預告）1,000 公尺以上，開發單位承諾退縮風場至距離中華白海豚重要棲息環境至少 4.2 公里、水深大於 37 公尺（TWVD2001 為基準）區域，於禁區邊界 4 個方位設置水下聲學監測設施，配備觀察船及配置鯨豚生態觀察員，於環境影響評估內容中，呈現最保守情境再加上極端氣候之最壞情境下評估，以及船隻船速管制在 6 節以下，且儘可能避免在中華白海豚活動高峰時間進入已知之中華白海豚活動密集位置，航道劃設也應避開敏感區位。第324次會議中，有哪些專家為鯨豚保育發言？，提出全程使用減噪措施如水下氣泡幕、水下帷幕或施工時已商業化之「最佳」水下噪音防制工法，以及施工營運階段鯨豚生態調查頻率應採每年 30 趟次（非僅限於 4~9 月執行，調整前應依法申請變更），建議強化鯨豚觀測員訓練作業，並考量邀民間團體具鯨豚觀測能力人員共同參與等環境及生態保育相關施加措施。\n\n第324次會議中，為鯨豚保育發言的專家有：1. 公部門的農業處研究專家、2.環境影響評估書件專家、3.彰化縣政府（農業處）研究專家、4.行政院農業委員會專家、5.王委員文誠專家等。'

In [96]:
final = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '請給我總結一下之下的內容，總結應該包括所有問題的答案，包括日期，還有專家是誰'+allQandA,
  temperature=0.9,
  max_tokens=1300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
final = final + text

final

'總結：第324次會議於107年1月17日（星期三）下午2時0分開始，為中華白海豚保育提出了多項措施，包括施工前使用聲音驅趕裝置，風機位置需距離野生動物1,000公尺以上，退縮風場至4.2公里、水深37公尺以上，於禁區邊界設置水下聲學監測設施，配備觀察船及鯨豚生態觀察員，以及提出減噪措施如水下氣泡幕、帷幕，每年調查頻率需至少為30次，考量邀民間團體具鯨豚觀測能力人員共同參與等環境及生態保育相關施加措施，專家有公部門的農業處研究專家、環境影響評估書件專家、彰化縣政府（農業處）研究專家、行政院農業委員會專家、王委員文誠專家等，仍懇請臺中市政府環境保護局歸還函上簽章確認。'

## (optional) adjust answers based on more information (not finished yet)

### prepare additional data

In [ ]:
new_df

In [ ]:
# 補充answer with other rows?
# reset index
new_df=new_df['text']
new_df = new_df.reset_index(drop=True)


In [ ]:
new_df=new_df.to_frame()

In [ ]:
df=new_df

In [ ]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 3600
index = df.index[df['cumulative_length'] > 3600][0]

# Create a new dataframe with only the last few rows
new_df = df.iloc[index:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index]

In [ ]:
df

In [ ]:
sbert_out_1=df['text']
sbert_out_1=sbert_out_1.to_frame()
sbert_out_1

In [ ]:
sbert_text_1 = sbert_out_1['text'].to_string(index=False)

In [ ]:
sbert_text_1 = re.sub(r'[\n\\]+', '', sbert_text_1)
sbert_text_1 = re.sub(r'n|n－', '', sbert_text_1)
sbert_text_1

### prompt GPT (part 2)